# Census Demo

In [1]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

import itertools
import time
from multiprocessing import Pool

import numpy as np
import pandas as pd

from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
import sklearn
import six
import verta
from verta.utils import ModelAPI
from verta._demo_utils import DeployedModel

/Users/conrado/workspace/modeldb-oss/client/verta/verta/_demo_utils.py:11: FutureWarning: `DeployedModel` is being moved to the `verta.deployment` module, and `verta._demo_utils` will be removed in a future version; consider using `from verta.deployment import DeployedModel` instead
  category=FutureWarning)


In [2]:
df_train = pd.read_csv("census-train.csv")
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:, -1]
df_test = pd.read_csv("census-test.csv")
X_test = df_test.iloc[:,:-1].values
hyperparam_candidates = {
    'C': [1e-4, 1e-6],
    'solver': ['lbfgs'],
    'max_iter': [28, 32],
}
hyperparam_sets = [dict(zip(hyperparam_candidates.keys(), values))
                   for values
                   in itertools.product(*hyperparam_candidates.values())]

# Log Workflow

## Connect to the platform

In [3]:
from verta import Client
client = Client("https://demo.dev.verta.ai")

set email from environment
set developer key from environment
connection successfully established


## Version a dataset

In [4]:
dataset = client.set_dataset(name="Census dataset - train", workspace="Demos")
dataset.create_version('small-census-train.csv')
train_version = dataset.create_version('census-train.csv')

dataset = client.set_dataset(name="Census dataset - test", workspace="Demos")
dataset.create_version('small-census-test.csv')
test_version = dataset.create_version('census-test.csv')

created new Dataset: Census dataset - train in workspace: Demos
created new DatasetVersion: 1
created new DatasetVersion: 2
created new Dataset: Census dataset - test in workspace: Demos
created new DatasetVersion: 1
created new DatasetVersion: 2


## Version a model

In [5]:
proj = client.set_project("Census - Demo", workspace="Demos")
expt = client.set_experiment("Linear regression")

created new Project: Census - Demo in workspace: Demos
created new Experiment: Linear regression


### Log the 4 components of a model

In [6]:
runs = []
for i, hyperparams in enumerate(hyperparam_sets):
    run = client.set_experiment_run("Iteration %d" % i)
    runs.append(run)
    
    # Config
    run.log_hyperparameters(hyperparams)
    # Code
    run.log_code()
    # Environment
    run.log_requirements(['scikit-learn'])
    # Dataset
    run.log_dataset_version('train', train_version)
    run.log_dataset_version('test', test_version)

created new ExperimentRun: Iteration 0
Git repository successfully located at /Users/conrado/workspace/modeldb-oss/
upload complete (requirements.txt)
created new ExperimentRun: Iteration 1
Git repository successfully located at /Users/conrado/workspace/modeldb-oss/
upload complete (requirements.txt)
created new ExperimentRun: Iteration 2
Git repository successfully located at /Users/conrado/workspace/modeldb-oss/
upload complete (requirements.txt)
created new ExperimentRun: Iteration 3
Git repository successfully located at /Users/conrado/workspace/modeldb-oss/
upload complete (requirements.txt)


In [5]:
print(run)

name: Iteration 3
url: https://demo.dev.verta.ai/Demos/projects/a1cad6e2-e55e-4a26-88ae-71d8642667ad/exp-runs/f188510a-c9c8-4f1c-9240-ce8ddef55ff6
description: 
tags: []
attributes: {}
id: f188510a-c9c8-4f1c-9240-ce8ddef55ff6
experiment id: 476f9fec-5ac1-4957-8a56-e25366887f86
project id: a1cad6e2-e55e-4a26-88ae-71d8642667ad
hyperparameters: {'C': 1e-06, 'solver': 'lbfgs', 'max_iter': 32}
observations: {}
metrics: {'val_acc': 0.7947761194029851}
artifact keys: ['requirements.txt', 'custom_modules', 'model.pkl', 'model_api.json', 'train_data']


### Train and log the results

In [13]:
(X_val_train, X_val_test,
 y_val_train, y_val_test) = model_selection.train_test_split(X_train, y_train,
                                                             test_size=0.2,
                                                             shuffle=True)

model = linear_model.LogisticRegression(**hyperparams)
model.fit(X_train, y_train)

val_acc = model.score(X_val_test, y_val_test)
run.log_metric("val_acc", val_acc)

ValueError: metric with key val_acc already exists; consider using observations instead

### Log information for deployment

In [9]:
model_api = ModelAPI(X_train, model.predict(X_train))
run.log_model(model, model_api=model_api)
run.log_training_data(X_train, y_train)

upload complete (custom_modules.zip)
upload complete (model.pkl)
upload complete (model_api.json)
upload complete (train_data.csv)


## Deploy a model

In [8]:
deployed_model = run.get_deployed_model()
print(deployed_model)

<DeployedModel at https://demo.dev.verta.ai/api/v1/predict/f188510a-c9c8-4f1c-9240-ce8ddef55ff6>


In [9]:
for x in itertools.cycle(X_test):
    print(deployed_model.predict([x.tolist()]))
    time.sleep(0.1)

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]


[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[1]
[1]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]


KeyboardInterrupt: 